In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
dataset_path = 'D:/Proyectos_IA/Reconocimiento_Perros/dogImages'
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'valid')
test_dir = os.path.join(dataset_path, 'test')

In [4]:
#Estructura datos de entrada
IMG_SHAPE = (128, 128, 3)
#Uso de red neuronal preentrenada
model_preentrenado = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, 
                                                            include_top=False, 
                                                            weights='imagenet')
model_preentrenado.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 16, 16, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 16, 16, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 16, 16, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 16, 16, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 8, 8, 384)    0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 8, 8, 384)   3456        ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 8, 8, 384)   1536        ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 8, 8, 96)    384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 8, 8, 576)    55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 8, 8, 576)   2304        ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 8, 8, 576)    0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 4, 4, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 4, 4, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 4, 4, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 4, 4, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

In [5]:
model_preentrenado.trainable = False

#Configurando la capa de salida como la media de las salidas
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(model_preentrenado.output)


#Units = 133 porque son 133 razas y una activación por probabilidad de aparación
prediction_layer = tf.keras.layers.Dense(units=256, activation='softmax')(global_average_layer)
model = tf.keras.models.Model(inputs=model_preentrenado.input, outputs=prediction_layer)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
              loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [6]:
data_gen_train = ImageDataGenerator(rescale = 1/255.)  #rotation_range = 20, width_shift_range = 0.2, height_shift_range=0.2, zoom_range = 0.1
data_gen_valid = ImageDataGenerator(rescale = 1/255.)
data_gen_test = ImageDataGenerator(rescale = 1/255.)

In [7]:
batch_size_data_generator = 16
train_generator = data_gen_train.flow_from_directory(train_dir, target_size = (128, 128), 
                                                     batch_size = batch_size_data_generator, class_mode = 'sparse')

validation_generator = data_gen_valid.flow_from_directory(validation_dir, target_size = (128, 128),
                                                          batch_size = batch_size_data_generator, class_mode = 'sparse')

test_generator = data_gen_test.flow_from_directory(test_dir, target_size = (128, 128),
                                                          batch_size = batch_size_data_generator, class_mode = 'sparse')

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=6, min_delta=0.05,
                                                  restore_best_weights=True, mode = 'max')


history =  model.fit(x=train_generator, epochs=20, 
                     validation_data=validation_generator, batch_size = 16, 
                     callbacks=[early_stopping])

Epoch 1/20
418/418 [==============================] - 252s 593ms/step - loss: 3.2687 - sparse_categorical_accuracy: 0.5219 - val_loss: 3.2613 - val_sparse_categorical_accuracy: 0.5976
Epoch 2/20
418/418 [==============================] - 54s 128ms/step - loss: 1.4300 - sparse_categorical_accuracy: 0.7774 - val_loss: 3.4552 - val_sparse_categorical_accuracy: 0.6168
Epoch 3/20
418/418 [==============================] - 54s 129ms/step - loss: 0.7981 - sparse_categorical_accuracy: 0.8555 - val_loss: 3.4845 - val_sparse_categorical_accuracy: 0.6515
Epoch 4/20
418/418 [==============================] - 54s 130ms/step - loss: 0.4838 - sparse_categorical_accuracy: 0.9030 - val_loss: 3.6080 - val_sparse_categorical_accuracy: 0.6443
Epoch 5/20
418/418 [==============================] - 59s 142ms/step - loss: 0.2978 - sparse_categorical_accuracy: 0.9332 - val_loss: 3.5350 - val_sparse_categorical_accuracy: 0.6599
Epoch 6/20
418/418 [==============================] - 58s 139ms/step - loss: 0.1673 

In [9]:
evaluacion = model.evaluate(test_generator)

53/53 [==============================] - 22s 418ms/step - loss: 3.4263 - sparse_categorical_accuracy: 0.6603


In [10]:
#Fine Tuning para reentrenar el modelo despues de determinada capa
fine_tune_at = 90    #120

#Aquí se descongelan son las capas que tiene el modelo, en este caso se entrenaran las ultimas 35 capas de la red neuronal
#TENGA EN CUENTA QUE EL MODELO PREENTRENADO TIENE 155 CAPAS 
for layer in model_preentrenado.layers[:fine_tune_at]:
    layer.trainable = False

In [11]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

history =  model.fit(x=train_generator, epochs=12, 
                     validation_data=validation_generator, batch_size = 16, 
                     callbacks=[early_stopping])

Epoch 1/12
418/418 [==============================] - 57s 129ms/step - loss: 0.0257 - sparse_categorical_accuracy: 0.9921 - val_loss: 3.1491 - val_sparse_categorical_accuracy: 0.6862
Epoch 2/12
418/418 [==============================] - 53s 126ms/step - loss: 0.0065 - sparse_categorical_accuracy: 0.9982 - val_loss: 3.1303 - val_sparse_categorical_accuracy: 0.6922
Epoch 3/12
418/418 [==============================] - 55s 132ms/step - loss: 0.0051 - sparse_categorical_accuracy: 0.9985 - val_loss: 3.1007 - val_sparse_categorical_accuracy: 0.6934
Epoch 4/12
418/418 [==============================] - 57s 136ms/step - loss: 0.0047 - sparse_categorical_accuracy: 0.9987 - val_loss: 3.1006 - val_sparse_categorical_accuracy: 0.6886
Epoch 5/12
418/418 [==============================] - 55s 131ms/step - loss: 0.0042 - sparse_categorical_accuracy: 0.9988 - val_loss: 3.0932 - val_sparse_categorical_accuracy: 0.6946
Epoch 6/12
418/418 [==============================] - 53s 128ms/step - loss: 0.0037 -

In [12]:
evaluacion = model.evaluate(test_generator)

53/53 [==============================] - 6s 113ms/step - loss: 2.7448 - sparse_categorical_accuracy: 0.7081
